<a href="https://colab.research.google.com/github/Spica08/deep-learning-from-scratch-5/blob/main/step8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set up
import os
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# step8 拡散モデルの理論

## 8.1 VAEから拡散モデルへ
本書での「拡散モデル」は、「Denoising Diffusion Probabilistic Models」を指す。

### 8.1.1 VAEの復習
VAEでは、潜在変数を固定の正規分布よりサンプリングし、潜在変数から観測変数への変換をニューラルネットワークで行うことでデータ生成を行う。また、データxが得られた時の潜在変数zの事後分布を求めるためにもう1つ別のニューラルネットワークを使用する。

### 8.1.2 潜在変数の階層化
これまで見てきたVAEは潜在変数の数が1つだったが、この潜在変数を階層化したモデルを階層型VAEという。  
階層型VAEでは、直前の確率変数だけから決定される。この性質をマルコフ性といい、これを仮定することでパラメータの増加を防ぐ事ができる。そして潜在変数を階層化することでより複雑な表現を可能にする。  
例えば階層をT層にすることを考えると、それぞれの階層でencoder, decoderが必要となり合計2T個のニューラルネットワークが必要になる。このままではTが大きくなった時に実現が困難になってしまう。これを解決するために進化したものが拡散モデルである。

### 8.1.3 拡散モデルへ
階層型VAEは以下の2点を変更するだけで「拡散モデル」となる。  
1. 観測変数と潜在変数の次元を同じにする。  
2. エンコーダは、固定の正規分布によるノイズを追加する  

これに伴って、潜在変数の記号は全てxで統一し、エンコーダのパラメータが不要となる。そして、拡散モデルでは、ノイズを除去する過程をニューラルネットワークでモデル化する。

## 8.2 拡散過程と逆拡散過程
拡散モデルには、1:ノイズを追加する**拡散過程**と、2:ノイズを除去する**逆拡散過程**が存在する。

### 8.2.1 拡散過程
拡散過程では、1つ前の時刻のデータに対してノイズを加える。このノイズの加え方に関して満たすべき条件は、「最終時刻における潜在変数$x_T$が完全なノイズになること」である。  

ここで、$x_T$が完全なノイズ$\mathcal{N}(x_T;0,I)$に従うように、各時刻で加えるノイズの大きさを考える。1つに以下の方法がある。  
\begin{equation}
q(x_T | x_{T - 1}) = \mathcal{N}(x_T;\sqrt{1 - \beta_t}x_{T - 1}, \beta_tI)
\end{equation}
tは各時刻を表し、$\beta_t$はあらかじめ設定する0.01などの値とする。$\beta_t$が大きいほど分散と共に加えるノイズが大きくなる。この値を時刻ごとにT個の値として設定する。この時Tを1000くらいにある程度大きくし各βを適切に設定(ノイズスケジューリングという)すれば、最終的なデータは$\mathcal{N}(x_T;0,I)$に従う(後述)。

\begin{equation}
q(x_T | x_{T - 1}) = \mathcal{N}(x_T;\sqrt{1 - \beta_t}x_{T - 1}, \beta_tI)
\end{equation}
この式は、平均ベクトルが$\sqrt{1 - \beta_t}x_{T - 1}$、共分散行列が$\beta_tI$の正規分布であり、サンプリングされる値は変数変換トリックを用いると

\begin{align}
\epsilon &\backsim \mathcal{N}(\epsilon;0,I)\\
x_T &= \sqrt{1 - \beta_t}x_{T - 1} + \sqrt{\beta_T}\epsilon
\end{align}
と表す事ができる。これは、標準正規分布より$\epsilon$をサンプリングし、$\sqrt{\beta_T}$倍した値をノイズとして加えることを表す。つまり、前時刻のデータをややスケールダウンさせ、小さなノイズを加える、ということを繰り返す。

# 8.2.2 逆拡散過程
逆拡散過程ではノイズを除去する処理をニューラルネットワークで行う。  

拡散モデルでは全部でT回のノイズ除去を行う。各時刻において個別のニューラルネットワークを使用することも考えられるが、これではTが大きくなった時に計算が現実的ではなくなってしまう。  
拡散モデルでは、$\lbrace x_0, x_1, \dots, x_n\rbrace$の次元数が全て同じであるため、ニューラルネットワークの入出力の次元数も同じになる。これによりニューラルネットワークの構造を全て共通化できる。ここに、時刻tの情報を入力として与えることで、各時刻のノイズ除去を1つのニューラルネットワークで行う事ができる。  

以上より、逆拡散過程の目的は、ニューラルネットワークを用いて$p_\theta(x_{t - 1} | x_t)$をモデル化する事である。拡散過程が正規分布に従う場合、時間幅が十分小さければ$p_\theta(x_{t - 1} | x_t)$も正規分布に従うが、確率分布を直接出力することはでできないので、出力を平均ベクトルとする正規分布だと考えてネットワークを構成する。数式で表すと以下の通り。
\begin{align}
\hat{x}_{t - 1} &= NeuralNet(x_t, t;\theta)\qquad 時刻tのデータをニューラルネットワークへ入力し\\
p_\theta(x_{t - 1} | x_t) &= \mathcal{N}(x_{t - 1};\hat{x}_{t - 1},I) \qquad その出力を平均ベクトルとする正規分布から前の時刻のデータを得る。
\end{align}

## 8.3 ELBOの計算-1

### 8.3.1 拡散モデルのELBO
拡散モデルのELBOはVAEと同様に導出できる。VAEのELBOは以下。
\begin{align}
ELBO(x;\theta, \phi) &= \int q_\phi(z | x)log\frac{p_\theta(x, z)}{q_\phi(z | x)}dz\\
                     &= \mathbb{E}_{q_\phi(z | x)}\left[log\frac{p_\theta(x, z)}{q_\phi(z | x)}\right]
\end{align}
3点変更して拡散モデルのELBOとする。変更点は以下の通り。
1. $x$を$x_0$に変更  
2. z(潜在変数)を$\lbrace x_1, x_2, \dots, x_T\rbrace$へ変更  
3. パラメータ$\phi$(エンコーダのパラメータ)を消去

変更したELBOは以下の通り。
\begin{align}
ELBO(x;\theta, \phi) &= \mathbb{E}_{q(x_1, x_2, \dots, x_T | x_0)}\left[log\frac{p_\theta(x_0, x_1, x_2, \dots, x_T)}{q(x_1, x_2, \dots, x_T | x_0)}\right]\\
                     &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\frac{p_\theta(x_0:x_T)}{q(x_1:x_T | x_0)}\right]
\end{align}

### 8.3.2 ELBOの式展開
先ほど考えたELBOに出てくる$p_\theta(x_0:x_T)$について考えると、乗法定理とマルコフ性により以下のように変形できる。
\begin{align}
p_\theta(x_0:x_T) &= p_\theta(x_0|x_1)p_\theta(x_1|x_2)\dots p_\theta(x_{T - 1}|x_T)p(x_T)\\
                   &= p(x_T) \prod_{t = 1}^T p_\theta(x_{t - 1} | x_t)
\end{align}
同様に$q(x_1:x_T | x_0)$は、
\begin{align}
q(x_1:x_T | x_0) &= \prod_{t = 1}^T q(x_{t} | x_{t - 1})
\end{align}
となる。よってELBOは、
\begin{align}
ELBO(x;\theta, \phi) &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\frac{p_\theta(x_0:x_T)}{q(x_1:x_T | x_0)}\right]\\
                     &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\frac{p(x_T) \prod_{t = 1}^T p_\theta(x_{t - 1} | x_t)}{\prod_{t = 1}^T q(x_{t} | x_{t - 1})}\right]\\
                     &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\prod_{t = 1}^T p_\theta(x_{t - 1} | x_t) + log\frac{p(x_T)}{\prod_{t = 1}^T q(x_{t} | x_{t - 1})}\right]
\end{align}
$\theta$を含まない項のみを考慮し新たに目的関数$J(\theta)$とすると、
\begin{align}
J(\theta) &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\prod_{t = 1}^T p_\theta(x_{t - 1} | x_t)\right]\\
          &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[\sum_{t = 1}^T logp_\theta(x_{t - 1} | x_t)\right]
\end{align}
この目的関数は期待値で表されるので、モンテカルロ法で近似する。サンプルサイズを1とすると、
\begin{align}
x_1:x_T &\backsim q(x_1:x_T | x_0)\\
J(\theta) &\approx \sum_{t = 1}^T logp_\theta(x_{t - 1} | x_t)
\end{align}
この時の$P_\theta(x_{t - 1}|x_t)$は、
\begin{align}
\hat{x}_{t - 1} &= NeuralNet(x_t, t;\theta)\\
p_\theta(x_{t - 1} | x_t) &= \mathcal{N}(x_{t - 1};\hat{x}_{t - 1},I)
\end{align}
よって目的関数は、
\begin{align}
J(\theta) &\approx \sum_{t = 1}^T logp_\theta(x_{t - 1} | x_t)\\
          &= \sum_{t = 1}^T log\mathcal{N}(x_{t - 1};\hat{x}_{t - 1},I)\\
          &= \sum_{t = 0}^{T - 1} log\mathcal{N}(x_t;\hat{x}_t,I)\\
          &= \sum_{t = 0}^{T - 1} log \left(\frac{1}{\sqrt{(2\pi)^D|I|}}exp\left(-\frac{1}{2}(x_t - \hat{x_t})^TI^{-1}(x_t - \hat{x_t}) \right)\right)\\
          &= \sum_{t = 0}^{T - 1} \left(-\frac{1}{2}(x_t - \hat{x_t})^TI^{-1}(x_t - \hat{x_t}) \right) + \sum_{t = 0}^{T - 1} log\left(\frac{1}{\sqrt{(2\pi)^D|I|}}\right)\\
          &= -\frac{1}{2}\sum_{t = 0}^{T - 1}\left(x_t - \hat{x_t})^T(x_t - \hat{x_t}\right) + Tlog\left(\frac{1}{\sqrt{(2\pi)^D|I|}}\right)
\end{align}
定数項を無視して、
\begin{align}
J(\theta) &= -\frac{1}{2}\sum_{t = 0}^{T - 1}\left(x_t - \hat{x_t})^T(x_t - \hat{x_t}\right)\\
          &= -\frac{1}{2}\sum_{t = 0}^{T - 1} ||x_t - \hat{x_t}||^2
\end{align}

以上より、拡散モデルのELBOから損失関数を導出すると、  
1. 拡散過程によりT個のサンプリングを行い  
2. ニューラルネットワークをT回適用してノイズ除去を行い  
3. 各時刻における潜在変数とサンプリング結果との2乗誤差を求める。  

この方法では、拡散過程をT回行うときサンプリングもT回行う必要があり、計算量が大きくなるという課題がある。


## 8.4 ELBOの計算-2
前節では、ELBOをT個のサンプリングデータによって近似した。このままでは計算量が非常に大きいため、続いて2個のサンプリングデータで近似する方法について考える。  

ここで重要になるのが、拡散過程で$q(x_t | x_0)$が解析的に表せるということである。つまり、元データ$x_0$に対してノイズを一度だけ追加することで任意の時間tにおける$x_t$をサンプリングする事ができる。

### 8.4.1 $q(x_t | x_0)$の式
\begin{equation}
q(x_T | x_{T - 1}) = \mathcal{N}(x_T;\sqrt{1 - \beta_t}x_{T - 1}, \beta_tI)
\end{equation}
先ほどまでは上記の式に従って拡散過程ではT回の処理を行なっていた。これを1度の処理で済むように$q(x_t | x_0)$を解析的に表す必要がある。  

結果を先に示すと、
\begin{equation}
q(x_t | x_0) = \mathcal{N}(x_t;\sqrt{\bar{\alpha_t}}x_{0}, (1 - \bar{\alpha}_t)I)
\end{equation}
\begin{align}
\alpha_t &= 1 - \beta_t\\
\bar{\alpha_t} &= \alpha_t \alpha_{t - 1} \dots \alpha_1
\end{align}
となる。この式から、時刻Tにおける$x_T$が完全なノイズになることも確認する事ができる。

### 8.4.2 ELBOの近似解
前節までで考えてたELBOおよび目的関数は以下の通り。
\begin{align}
ELBO(x;\theta, \phi) &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\frac{p_\theta(x_0:x_T)}{q(x_1:x_T | x_0)}\right]\\
                     &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\frac{p(x_T) \prod_{t = 1}^T p_\theta(x_{t - 1} | x_t)}{\prod_{t = 1}^T q(x_{t} | x_{t - 1})}\right]\\
                     &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\prod_{t = 1}^T p_\theta(x_{t - 1} | x_t) + log\frac{p(x_T)}{\prod_{t = 1}^T q(x_{t} | x_{t - 1})}\right]\\
J(\theta) &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[log\prod_{t = 1}^T p_\theta(x_{t - 1} | x_t)\right]\\
          &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[\sum_{t = 1}^T logp_\theta(x_{t - 1} | x_t)\right]
\end{align}
この目的関数は以下のように展開する事ができる。
\begin{align}
J(\theta) &= \mathbb{E}_{q(x_1:x_T | x_0)}\left[\sum_{t = 1}^T logp_\theta(x_{t - 1} | x_t)\right]\\
          &= \sum_{t = 1}^T \mathbb{E}_{q(x_1:x_T | x_0)}\left[logp_\theta(x_{t - 1} | x_t)\right]\qquad \because 期待値の線形性\\
          &= \sum_{t = 1}^T \mathbb{E}_{q(x_{t - 1}, x_t | x_0)}\left[logp_\theta(x_{t - 1} | x_t)\right]\qquad \because 関連する変数の期待値\\
\end{align}
$q(x_{t - 1}, x_t | x_0)$からのサンプリングは2つの手順で生成する事ができる(1 -> $q(x_{t - 1} | x_0)$から$x_{t - 1}$をサンプリング、2 -> $q(x_t | x_{t - 1})$から$x_t$をサンプリング)。  

このままでは和の計算を解消しきれていないが、$\sum$を前に出したことによって和の計算を一様分布に関する期待値として表す事ができるようになった。  

一様分布$u(t)$について、任意の関数f(x)の期待値を考えると、
\begin{align}
\mathbb{E}_{u(t)}[f(x)] &= \sum_{t = 1}^T u(t)f(t)\\
                       &= \frac{1}{T}\sum_{t = 1}^Tf(t)\\
よって\\
\sum_{t = 1}^Tf(t) &= T\mathbb{E}_{u(t)}[f(t)]
\end{align}
以上より、任意の関数f(t)について1~TのT個の和は、一様分布$u(t)$に関する期待値として表す事ができる。先ほどの目的関数にこれを適用して、
\begin{align}
J(\theta) &= \sum_{t = 1}^T \mathbb{E}_{q(x_{t - 1}, x_t | x_0)}\left[logp_\theta(x_{t - 1} | x_t)\right]\\
          &= T\mathbb{E}_{u(t)}[\mathbb{E}_{q(x_{t - 1}, x_t | x_0)}\left[logp_\theta(x_{t - 1} | x_t)\right]]
\end{align}
目的関数を期待値をして表す事ができたので、モンテカルロ法による近似を行う事ができ和の計算の必要がなくなった。具体的には以下のように近似を行う。  

\begin{align}
t &\backsim \mathcal{U}\lbrace1, T\rbrace\\
x_{t - 1} &\backsim q(x_{t - 1} | x_0)\\
x_t &\backsim q(x_t | x_{t - 1})\\
J(\theta) &\approx Tlogp_\theta(x_{t - 1} | x_t)\\
          &= -\frac{T}{2} ||x_{t - 1} - \hat{x_{t - 1}}||^2
\end{align}

以上より、2回のサンプリングを利用した目的関数の計算では、
1. 一様分布から任意の時刻tをサンプリングする  
2. 2段階のサンプリングにより$x_{t - 1}, x_t$をサンプリングする
2. ニューラルネットワークに$x_t$を入力して$x_{t - 1}$を出力する  
3. 時刻t - 1における潜在変数とサンプリング結果との2乗誤差を求める。  

### 8.4.3 $q(x_t | x_0)$の導出
拡散過程では元データに対してガウスノイズをT回付与する。ここでガウスノイズの重要な性質として、「ガウスノイズの和もガウスノイズになる」というものがある。  
具体的には、$\mathcal{N_x}(\mu_x, \sigma_x^2), \mathcal{N_y}(\mu_y, \sigma_y^2)$があるとき、独立にサンプリングした変数x, yの和をzとすると、zは平均$\mu_x + \mu_y$、分散$\sigma_x^2 + \sigma_y^2$の正規分布に従う。  
これを拡散過程に適用する。拡散過程の各時刻でのノイズは以下
\begin{equation}
q(x_t | x_{t - 1}) = \mathcal{N}(x_t;\sqrt{1 - \beta_t}x_{t - 1}, \beta_tI)
\end{equation}
$\alpha_t = 1 - \beta_t$とすると、
\begin{equation}
q(x_t | x_{t - 1}) = \mathcal{N}(x_t;\sqrt{\alpha_t}x_{t - 1}, (1 - \alpha_t)I)
\end{equation}
このサンプリングを変数変換トリックを使用して表すと、
\begin{align}
\epsilon_t &\backsim \mathcal{N}(\epsilon_t;0, I)\\
x_t &= \sqrt{\alpha_t}x_{t - 1} + \epsilon_t \sqrt{1 - \alpha_t}
\end{align}
時刻t - 1を考えるためにtにt - 1を代入して、
\begin{align}
\epsilon_{t - 1} &\backsim \mathcal{N}(\epsilon_{t - 1};0, I)\\
x_{t - 1} &= \sqrt{\alpha_{t - 1}}x_{t - 2} + \epsilon_{t - 1} \sqrt{1 - \alpha_{t - 1}}
\end{align}
これらを合わせて以下の式を得る。
\begin{align}
x_t &= \sqrt{\alpha_t}(\sqrt{\alpha_{t - 1}}x_{t - 2} + \epsilon_{t - 1} \sqrt{1 - \alpha_{t - 1}}) + \epsilon_t \sqrt{1 - \alpha_t}\\
    &= \sqrt{\alpha_t}\sqrt{\alpha_{t - 1}}x_{t - 2} + \sqrt{\alpha_t}\epsilon_{t - 1} \sqrt{1 - \alpha_{t - 1}} + \epsilon_t \sqrt{1 - \alpha_t}\\
    &= \sqrt{\alpha_t}\sqrt{\alpha_{t - 1}}x_{t - 2} + \sqrt{\alpha_t - \alpha_t\alpha_{t - 1}} \epsilon_{t - 1} + \sqrt{1 - \alpha_t}\epsilon_t
\end{align}
$\epsilon_t, \epsilon_{t - 1}$は共に正規分布から独立に生成されたサンプルであるため、$\sqrt{\alpha_t - \alpha_t\alpha_{t - 1}} \epsilon_{t - 1} + \sqrt{1 - \alpha_t}\epsilon_t$ も1つの正規分布として表す事ができる。$\epsilon_t, \epsilon_{t - 1}$の平均、分散をそれぞれ考慮すると、
\begin{align}
\sqrt{\alpha_t - \alpha_t\alpha_{t - 1}} \epsilon_{t - 1} + \sqrt{1 - \alpha_t}\epsilon_t &\backsim \mathcal{N}(0, (\sqrt{\alpha_t - \alpha_t\alpha_{t - 1}})^2I + (\sqrt{1 - \alpha_t})^2I)\\
&\backsim \mathcal{N}(0, (1 - \alpha_t \alpha_{t - 1})I)
\end{align}
そのため$x_t$のサンプリングはt - 2時点でのデータとパラメータを使用して
\begin{align}
\epsilon_t &\backsim \mathcal{N}(\epsilon_t;0, I)\\
x_t &= \sqrt{\alpha_t \alpha_{t - 1}}x_{t - 2} + \sqrt{1 - \alpha_t \alpha_{t - 1}}\epsilon_t
\end{align}
と表す事ができる。これを繰り返すと、
\begin{align}
q(x_t | x_0) &= \mathcal{N}(x_t;\sqrt{\bar{\alpha_t}}x_{0}, (1 - \bar{\alpha}_t)I)\\
\alpha_t &= 1 - \beta_t\\
\bar{\alpha_t} &= \alpha_t \alpha_{t - 1} \dots \alpha_1
\end{align}
が得られる。

## 8.5 ELBOの計算-3
最後の改善として1つのサンプリングからELBOを近似する方法を導出する。  
重要となるのが$q(x_{t - 1}|x_t, x_0)$という確率分布で、これは解析的に表すことが可能である。

### 8.5.1 $q(x_{t - 1} |x_t, x_0)$の式(結果)
先ほど同様先に結果を示すと、
\begin{equation}
q(x_{t - 1} |x_t, x_0) = \mathcal{N}(x_{t - 1};\mu_q(x_t, x_0), \sigma_q^2(t)I)
\end{equation}

\begin{align}
\alpha_t &= 1 - \beta_t\\
\bar{\alpha_t} &= \alpha_t \alpha_{t - 1} \dots \alpha_1\\
\mu_q(x_t, x_0) &= \frac{\sqrt{\alpha_t}(1 - \bar{\alpha}_{t - 1})x_t + \sqrt{\bar{\alpha}_{t - 1}}(1 - \alpha_t)x_0}{1 - \bar{\alpha_t}}\\
\sigma_q^2(t) &= \frac{(1 - \alpha_t)(1 - \bar{\alpha}_{t - 1})}{1 - \bar{\alpha_t}}
\end{align}